In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

In [2]:
from llama_index import SimpleDirectoryReader, LLMPredictor
from llama_index.indices.knowledge_graph.base import GPTKnowledgeGraphIndex
from langchain import OpenAI
from IPython.display import Markdown, display

In [3]:
documents = SimpleDirectoryReader('/home/breadchris/notes/journals').load_data()

In [3]:
from langchain.chat_models import ChatOpenAI

In [4]:
llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"))

In [7]:
index = GPTKnowledgeGraphIndex(
    documents, 
    chunk_size_limit=512, 
    max_triplets_per_chunk=2,
    llm_predictor=llm_predictor
)

INFO:llama_index.token_counter.token_counter:> [build_index_from_documents] Total LLM token usage: 165681 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_documents] Total embedding token usage: 0 tokens


In [8]:
index.save_to_disk('index_kg.json')

In [5]:
new_index = GPTKnowledgeGraphIndex.load_from_disk('index_kg.json', llm_predictor=llm_predictor)

In [10]:
response = new_index.query(
    "What does lunasec do as a company?", 
    include_text=False, 
    response_mode="tree_summarize"
)

INFO:llama_index.indices.query.knowledge_graph.query:> Starting query: What does lunasec do as a company?
INFO:llama_index.indices.query.knowledge_graph.query:> Query keywords: ['company', 'lunasec']
ERROR:llama_index.indices.query.knowledge_graph.query:Index was not constructed with embeddings, skipping embedding usage...
INFO:llama_index.indices.query.knowledge_graph.query:> Extracted relationships: The following are knowledge triplets in the form of (subset, predicate, object):
('lunasec', 'has', 'install page')
('lunasec', 'talk about', 'epss')
('lunasec', 'take', 'contextual vulnerability data')
('lunasec', 'work separate out', 'scraper')
('lunasec', 'provides', 'vulnerability details')
('lunasec', 'inputting', 'CVE name')
INFO:llama_index.indices.common.tree.base:> Building index from nodes: 0 chunks
INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 485 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 0 tokens


In [11]:
display(Markdown(f"<b>{response}</b>"))

<b>Lunasec is a company that works in the field of cybersecurity and provides services related to vulnerability detection and management. They provide vulnerability details, work on separating out scraper, take contextual vulnerability data, talk about EPSS, have an install page, and allow inputting CVE name.</b>

In [6]:
from pyvis.network import Network

g = new_index.get_networkx_graph()
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(g)
net.show("example.html")

example.html
